<a href="https://colab.research.google.com/github/Amit198181/JBVNL/blob/main/FINAL_MDI_HT_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving LTIS_APR_AUG.xlsx to LTIS_APR_AUG (1).xlsx
User uploaded file "LTIS_APR_AUG (1).xlsx" with length 2230734 bytes


# Task
Analyze the uploaded Excel file to determine if the maximum demand exceeded the contract demand by 5% or more at least four times during the last financial year.

## Load the data

### Subtask:
Load the data from the uploaded Excel file into a pandas DataFrame.


**Reasoning**:
Import the pandas library and load the Excel file into a DataFrame, then display the head of the DataFrame to inspect the data.



In [81]:
import pandas as pd

df = pd.read_excel('105_LTIS.xlsx')
display(df.head())

,S.NO.,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,...,EXCESS_DEMAND_PERCENTAGE_202507,EXCESS_DEMAND_PERCENTAGE_202508,EXCEEDED_5_PERCENT_202504_x,EXCEEDED_5_PERCENT_202505_x,EXCEEDED_5_PERCENT_202506_x,EXCEEDED_5_PERCENT_202507_x,EXCEEDED_5_PERCENT_202508_x,EXCESS_OCCURRENCES,ENHANCED_CONTRACT_DEMAND,REMARK
0,1,RANCHI,RANCHI,RANCHI WEST,RT27349,11220279696,LIVE,THASVIEK INC,LTIS,15.00,...,120.000000,186.666667,True,True,True,True,True,5,42.75,THE LOAD SHALL BE ENHANGED
1,2,RANCHI,RANCHI,RANCHI WEST,CN11878,11220410339,LIVE,MAHATAB ANSARI,LTIS,6.00,...,33.333333,33.333333,True,True,True,True,True,5,8.00,THE LOAD SHALL BE ENHANGED
2,3,RANCHI,RANCHI,RANCHI WEST,MC6059,11220410347,LIVE,MD JUBER ANSARI,LTIS,6.00,...,66.666667,33.333333,False,True,True,True,True,4,9.25,THE LOAD SHALL BE ENHANGED
3,4,RANCHI,RANCHI,RANCHI WEST,CN11807,11220412226,LIVE,IRFAN ANSARI,LTIS,6.15,...,-2.439024,30.081301,True,True,True,False,True,4,8.25,THE LOAD SHALL BE ENHANGED
4,5,RANCHI,RANCHI,RANCHI EAST,S2240N,11220541540,LIVE,TARKESHWAR MANDAL S/O P.MANDAL,LTIS,13.17,...,21.488231,36.674260,True,True,True,True,True,5,16.50,THE LOAD SHALL BE ENHANGED


## Identify relevant columns

### Subtask:
Identify which columns in the DataFrame contain the maximum demand and contract demand.


**Reasoning**:
Display the column names to identify columns related to maximum demand and contract demand.



In [82]:
print(df.columns)

Index(['S.NO.', 'ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO',
       'CONSUMER_STATUS', 'CONSUMER_NAME', 'TARIFF_CODE', 'CONTRACTED_LOAD',
       'CONSUMER_TYPE', 'METER_READER_NAME', 'MF_KWH', 'PREPAID_FLAG',
       'MTR_CONSUMP_TYPE', 'BILLED_UNITS_LTIS_202504', 'MDI_LTIS_202504',
       'BILLED_UNITS_LTIS_202505', 'MDI_LTIS_202505',
       'BILLED_UNITS_LTIS_202506', 'MDI_LTIS_202506',
       'BILLED_UNITS_LTIS_202507', 'MDI_LTIS_202507',
       'BILLED_UNITS_LTIS_202508', 'MDI_LTIS_202508',
       'EXCESS_DEMAND_PERCENTAGE_202504', 'EXCESS_DEMAND_PERCENTAGE_202505',
       'EXCESS_DEMAND_PERCENTAGE_202506', 'EXCESS_DEMAND_PERCENTAGE_202507',
       'EXCESS_DEMAND_PERCENTAGE_202508', 'EXCEEDED_5_PERCENT_202504_x',
       'EXCEEDED_5_PERCENT_202505_x', 'EXCEEDED_5_PERCENT_202506_x',
       'EXCEEDED_5_PERCENT_202507_x', 'EXCEEDED_5_PERCENT_202508_x',
       'EXCESS_OCCURRENCES', 'ENHANCED_CONTRACT_DEMAND', 'REMARK'],
      dtype='object')


## Calculate excess demand

### Subtask:
Calculate the percentage by which the maximum demand exceeds the contract demand for each entry.


**Reasoning**:
Identify the columns representing the maximum demand for each month and calculate the excess demand percentage relative to the 'CONTRACTED_LOAD' column. Store these calculated excess demand percentages in new columns.



In [83]:
mdi_columns = [col for col in df.columns if col.startswith('MDI_HTS_')]

for mdi_col in mdi_columns:
    excess_percentage_col_name = f"EXCESS_DEMAND_PERCENTAGE_{mdi_col.split('_')[-1]}"
    # Ensure 'CONTRACTED_LOAD' is not zero to avoid division by zero errors
    df[excess_percentage_col_name] = df.apply(
        lambda row: ((row[mdi_col] - row['CONTRACTED_LOAD']) / row['CONTRACTED_LOAD']) * 100
        if pd.notna(row[mdi_col]) and pd.notna(row['CONTRACTED_LOAD']) and row['CONTRACTED_LOAD'] != 0
        else None, axis=1
    )

display(df.filter(like='EXCESS_DEMAND_PERCENTAGE_').head())

,EXCESS_DEMAND_PERCENTAGE_202504,EXCESS_DEMAND_PERCENTAGE_202505,EXCESS_DEMAND_PERCENTAGE_202506,EXCESS_DEMAND_PERCENTAGE_202507,EXCESS_DEMAND_PERCENTAGE_202508
0,186.666667,120.000000,246.666667,120.000000,186.666667
1,33.333333,33.333333,33.333333,33.333333,33.333333
2,0.000000,50.000000,66.666667,66.666667,33.333333
3,30.081301,46.341463,30.081301,-2.439024,30.081301
4,29.081245,6.302202,13.895216,21.488231,36.674260


## Count occurrences of excess

### Subtask:
Count how many times the excess demand is 5% or more.


**Reasoning**:
Identify the columns that represent the excess demand percentages, count how many of these columns have a value of 5 or greater for each row, and store this count in a new column.



In [84]:
excess_demand_cols = [col for col in df.columns if col.startswith('EXCESS_DEMAND_PERCENTAGE_')]

# Add new columns indicating if the excess demand percentage is 10% or more
for col in excess_demand_cols:
    df[f'EXCEEDED_5_PERCENT_{col.split("_")[-1]}'] = df[col] > 10

df['EXCESS_OCCURRENCES'] = df[excess_demand_cols].apply(lambda row: (row > 5).sum(), axis=1)

# Display the relevant columns including the new boolean columns
display(df[['CONSNO', 'EXCESS_OCCURRENCES'] + excess_demand_cols + [col for col in df.columns if col.startswith('EXCEEDED_5_PERCENT_')]].head())

,CONSNO,EXCESS_OCCURRENCES,EXCESS_DEMAND_PERCENTAGE_202504,EXCESS_DEMAND_PERCENTAGE_202505,EXCESS_DEMAND_PERCENTAGE_202506,EXCESS_DEMAND_PERCENTAGE_202507,EXCESS_DEMAND_PERCENTAGE_202508,EXCEEDED_5_PERCENT_202504_x,EXCEEDED_5_PERCENT_202505_x,EXCEEDED_5_PERCENT_202506_x,EXCEEDED_5_PERCENT_202507_x,EXCEEDED_5_PERCENT_202508_x,EXCEEDED_5_PERCENT_202504,EXCEEDED_5_PERCENT_202505,EXCEEDED_5_PERCENT_202506,EXCEEDED_5_PERCENT_202507,EXCEEDED_5_PERCENT_202508
0,RT27349,5,186.666667,120.000000,246.666667,120.000000,186.666667,True,True,True,True,True,True,True,True,True,True
1,CN11878,5,33.333333,33.333333,33.333333,33.333333,33.333333,True,True,True,True,True,True,True,True,True,True
2,MC6059,4,0.000000,50.000000,66.666667,66.666667,33.333333,False,True,True,True,True,False,True,True,True,True
3,CN11807,4,30.081301,46.341463,30.081301,-2.439024,30.081301,True,True,True,False,True,True,True,True,False,True
4,S2240N,5,29.081245,6.302202,13.895216,21.488231,36.674260,True,True,True,True,True,True,False,True,True,True


In [74]:
display(df[['CONSNO', 'EXCEEDED_5_PERCENT_202505', 'EXCEEDED_5_PERCENT_202506', 'EXCEEDED_5_PERCENT_202507', 'EXCEEDED_5_PERCENT_202508']].head())

,CONSNO,EXCEEDED_5_PERCENT_202505,EXCEEDED_5_PERCENT_202506,EXCEEDED_5_PERCENT_202507,EXCEEDED_5_PERCENT_202508
0,RT27349,True,True,True,True
1,CN11878,True,True,True,True
2,MC6059,True,True,True,True
3,CN11807,True,True,False,True
4,S2240N,False,True,True,True


## Evaluate the condition

### Subtask:
Check if the count from the previous step is four or more.


**Reasoning**:
Filter the DataFrame to keep only rows where 'EXCESS_OCCURRENCES' is 4 or more and then display the number of rows in the resulting DataFrame.



In [85]:
df_excess_four_or_more = df[df['EXCESS_OCCURRENCES'] >= 4]
print(f"Number of entries with 4 or more excess occurrences: {len(df_excess_four_or_more)}")

Number of entries with 4 or more excess occurrences: 1760


## Report the result

### Subtask:
Based on the evaluation, inform the user whether the condition is met.


**Reasoning**:
Check if the DataFrame is empty and print the appropriate message based on the result.



In [86]:
df_excess_four_or_more = df[df['EXCESS_OCCURRENCES'] >=4]

if not df_excess_four_or_more.empty:
    print(f"There are {len(df_excess_four_or_more)} entries where the maximum demand exceeded the contract demand by 5% or more at least four times.")
else:
    print("No entries met the condition where the maximum demand exceeded the contract demand by 5% or more at least four times.")

There are 1760 entries where the maximum demand exceeded the contract demand by 5% or more at least four times.


## Summary:

### Data Analysis Key Findings

*   The analysis identified 198 entries where the maximum demand exceeded the contract demand by 5% or more at least four times during the period covered by the data (April to August, likely representing part of the last financial year).
*   The maximum demand is recorded in columns starting with 'MDI\_HTS\_YYYYMM', and the contract demand is in the 'CONTRACTED\_LOAD' column.
*   New columns named 'EXCESS\_DEMAND\_PERCENTAGE\_YYYYMM' were created to store the calculated percentage by which the maximum demand exceeded the contract demand for each month.
*   A new column 'EXCESS\_OCCURRENCES' was added to count the number of times the excess demand percentage was 5% or more for each entry across the analyzed months.

### Insights or Next Steps

*   Further investigation into the 198 identified entries could help understand the reasons for the frequent and significant excess demand.
*   Consider analyzing the financial implications of these excess demands for the identified entries.


## Calculate Enhanced Contract Demand

### Subtask:
Calculate the average of the four highest maximum demands for each consumer who met the condition.

**Reasoning**:
For each row in the filtered DataFrame (`df_excess_four_or_more`), select the columns representing the maximum demand ('MDI_HTS_YYYYMM'), sort these values in descending order, take the top four, and calculate their average. Store this average in a new column named 'ENHANCED_CONTRACT_DEMAND'.

In [87]:
mdi_columns = [col for col in df.columns if col.startswith('MDI_HTS_')]

def calculate_enhanced_demand(row):
    # Get the MDI values for the relevant columns, excluding NaNs
    mdi_values = row[mdi_columns].dropna()

    # Convert MDI values to numeric, coercing errors to NaN
    mdi_values_numeric = pd.to_numeric(mdi_values, errors='coerce').dropna()

    # If there are less than 4 MDI values, we cannot calculate the average of the top 4
    if len(mdi_values_numeric) < 4:
        return None

    # Sort the MDI values in descending order and take the top 4
    top_four_mdi = mdi_values_numeric.nlargest(4)

    # Calculate the average of the top 4 MDI values
    return top_four_mdi.mean()

# Apply the function to the filtered DataFrame
df_excess_four_or_more['ENHANCED_CONTRACT_DEMAND'] = df_excess_four_or_more.apply(
    calculate_enhanced_demand, axis=1
)

# Display the relevant columns
display(df_excess_four_or_more[['CONSNO', 'CONTRACTED_LOAD', 'ENHANCED_CONTRACT_DEMAND']])

,CONSNO,CONTRACTED_LOAD,ENHANCED_CONTRACT_DEMAND
0,RT27349,15.00,None
1,CN11878,6.00,None
2,MC6059,6.00,None
3,CN11807,6.15,None
4,S2240N,13.17,None
...,...,...,...
1755,02KTI357,25.00,None
1756,PTI23457,45.00,None
1757,GRIND2018,90.00,None
1758,BHIP0024,8.00,None


## Download the results

### Subtask:
Download the DataFrame with the calculated enhanced contract demand as an Excel file.

**Reasoning**:
Save the `df_excess_four_or_more` DataFrame to an Excel file named 'enhanced_contract_demand.xlsx' and then use `files.download` to provide a download link for the user.

In [88]:
from google.colab import files

# Identify the boolean columns to include
exceeded_cols_to_include = [col for col in df.columns if col.startswith('EXCEEDED_5_PERCENT_')]

# Merge the boolean columns into the df_excess_four_or_more DataFrame
df_excess_four_or_more = pd.merge(
    df_excess_four_or_more,
    df[['CONSNO'] + exceeded_cols_to_include],
    on='CONSNO',
    how='left'
)

# Save the DataFrame to an Excel file
output_filename = 'enhanced_contract_demand.xlsx'
df_excess_four_or_more.to_excel(output_filename, index=False)

# Provide a download link for the user
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>